# 01. Gradient Descent & Optimizer 직접 구현

## 학습 목표
- Gradient Descent의 동작 원리 이해
- Momentum, RMSprop, Adam을 **직접 구현**
- 각 optimizer의 차이를 시각화로 확인

## 관련 면접 질문
- Gradient Descent에 대해서 쉽게 설명한다면?
- SGD, RMSprop, Adam에 대해서 아는대로 설명한다면?
- 모멘텀의 수식을 적어 본다면?
- Local Minima 문제에도 불구하고 딥러닝이 잘 되는 이유는?

---
## Pre-Quiz (구현 전에 먼저 답해보세요)

### Q1. Gradient Descent를 한 문장으로 설명한다면?
```
당신의 답: 
```

### Q2. Momentum이 필요한 이유는?
```
당신의 답: 
```

### Q3. Adam = Momentum + RMSprop인데, 각각 무엇을 담당하나요?
```
Momentum이 담당하는 것: 
RMSprop이 담당하는 것: 
```

---
## Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 한글 폰트 설정 (macOS)
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (10, 6)

---
## 1. 손실 함수 정의 (제공됨)

최적화 테스트용 함수들입니다. 이 부분은 그대로 사용하세요.

In [ ]:
def quadratic(x, y):
    """간단한 이차 함수: f(x,y) = x² + 5y²
    
    특징: y 방향이 x 방향보다 5배 가파름 → 지그재그 현상 발생
    Global minimum: (0, 0)
    """
    return x**2 + 5*y**2

def quadratic_gradient(x, y):
    """이차 함수의 gradient"""
    return np.array([2*x, 10*y])


def rosenbrock(x, y):
    """Rosenbrock 함수: 좁은 골짜기 형태
    
    Global minimum: (1, 1) = 0
    난이도 높음: 골짜기를 따라가야 함
    """
    return (1 - x)**2 + 100*(y - x**2)**2

def rosenbrock_gradient(x, y):
    """Rosenbrock 함수의 gradient"""
    dx = -2*(1 - x) - 400*x*(y - x**2)
    dy = 200*(y - x**2)
    return np.array([dx, dy])

---
## 2. Vanilla Gradient Descent 구현

가장 기본적인 GD입니다.

### 수식
$$\theta_{t+1} = \theta_t - \eta \cdot \nabla L$$

- $\eta$: learning rate
- $\nabla L$: gradient

In [ ]:
# =================================================
# TODO: Vanilla Gradient Descent 구현
# =================================================
#
# 수식: θ_new = θ - lr * gradient
#
# 요구사항:
#   - __init__(self, lr): learning rate 저장
#   - update(self, params, grads): 업데이트된 params 반환
#
# =================================================

class VanillaGD:
    def __init__(self, lr=0.01):
        # TODO: learning rate 저장
        pass
    
    def update(self, params, grads):
        # TODO: params 업데이트 후 반환
        # 힌트: params - lr * grads
        pass

In [ ]:
# ============ 테스트 ============
def test_vanilla_gd():
    opt = VanillaGD(lr=0.1)
    params = np.array([1.0, 2.0])
    grads = np.array([0.5, 0.5])
    
    new_params = opt.update(params, grads)
    expected = np.array([0.95, 1.95])  # 1 - 0.1*0.5, 2 - 0.1*0.5
    
    assert new_params is not None, "update 메서드 구현 필요"
    assert np.allclose(new_params, expected), f"Expected {expected}, got {new_params}"
    print("✅ VanillaGD 테스트 통과!")

test_vanilla_gd()

---
## 3. Momentum 구현

### 핵심 아이디어
- 과거 gradient 방향을 **기억**하여 관성처럼 작용
- 지그재그 현상 감소, 수렴 속도 향상

### 수식
$$v_t = \beta \cdot v_{t-1} + \eta \cdot \nabla L$$
$$\theta_{t+1} = \theta_t - v_t$$

- $\beta$: momentum 계수 (보통 0.9)
- $v_t$: velocity (속도)

In [ ]:
# =================================================
# TODO: Momentum SGD 구현
# =================================================
#
# 수식:
#   v = β * v + lr * gradient
#   θ_new = θ - v
#
# 요구사항:
#   - __init__(self, lr, momentum): lr, momentum(β) 저장, velocity 초기화
#   - update(self, params, grads): velocity 업데이트 후 params 반환
#
# 주의:
#   - velocity는 처음에 None으로 두고, 첫 update에서 초기화
#   - velocity shape은 params와 동일해야 함
#
# =================================================

class MomentumGD:
    def __init__(self, lr=0.01, momentum=0.9):
        # TODO: lr, momentum 저장, velocity는 None으로 초기화
        pass
    
    def update(self, params, grads):
        # TODO:
        # 1. velocity가 None이면 zeros_like(params)로 초기화
        # 2. velocity 업데이트: v = β * v + lr * grads
        # 3. params 업데이트: params - velocity
        pass

In [ ]:
# ============ 테스트 ============
def test_momentum():
    opt = MomentumGD(lr=0.1, momentum=0.9)
    params = np.array([1.0, 2.0])
    grads = np.array([0.5, 0.5])
    
    # 첫 번째 업데이트
    new_params = opt.update(params, grads)
    assert new_params is not None, "update 메서드 구현 필요"
    
    # 두 번째 업데이트 (momentum 효과 확인)
    new_params2 = opt.update(new_params, grads)
    
    # velocity가 누적되어야 함
    print(f"1st update: {new_params}")
    print(f"2nd update: {new_params2}")
    print("✅ MomentumGD 테스트 통과!")

test_momentum()

---
## 4. RMSprop 구현

### 핵심 아이디어
- gradient의 **크기(제곱)**를 누적
- 많이 업데이트된 방향은 lr 감소, 적게 업데이트된 방향은 lr 유지
- **Adaptive Learning Rate**

### 수식
$$E[g^2]_t = \gamma \cdot E[g^2]_{t-1} + (1-\gamma) \cdot g_t^2$$
$$\theta_{t+1} = \theta_t - \frac{\eta}{\sqrt{E[g^2]_t + \epsilon}} \cdot g_t$$

- $\gamma$: decay rate (보통 0.99)
- $\epsilon$: 0으로 나누기 방지 (보통 1e-8)

In [ ]:
# =================================================
# TODO: RMSprop 구현
# =================================================
#
# 수식:
#   cache = γ * cache + (1-γ) * grads²
#   θ_new = θ - lr * grads / (√cache + ε)
#
# 요구사항:
#   - __init__(self, lr, decay, epsilon): 파라미터 저장, cache는 None
#   - update(self, params, grads): cache 업데이트 후 params 반환
#
# =================================================

class RMSprop:
    def __init__(self, lr=0.01, decay=0.99, epsilon=1e-8):
        # TODO: lr, decay, epsilon 저장, cache는 None
        pass
    
    def update(self, params, grads):
        # TODO:
        # 1. cache가 None이면 zeros_like(params)로 초기화
        # 2. cache 업데이트: decay * cache + (1-decay) * grads**2
        # 3. params 업데이트: params - lr * grads / (sqrt(cache) + epsilon)
        pass

In [ ]:
# ============ 테스트 ============
def test_rmsprop():
    opt = RMSprop(lr=0.1, decay=0.9, epsilon=1e-8)
    params = np.array([1.0, 2.0])
    grads = np.array([0.5, 1.0])  # y 방향 gradient가 더 큼
    
    new_params = opt.update(params, grads)
    assert new_params is not None, "update 메서드 구현 필요"
    
    # y 방향은 gradient가 커서 lr이 줄어들어야 함
    print(f"업데이트 후: {new_params}")
    print("✅ RMSprop 테스트 통과!")

test_rmsprop()

---
## 5. Adam 구현 (핵심!)

### 핵심 아이디어
**Adam = Momentum + RMSprop**
- 1차 모멘트 (m): gradient의 이동 평균 → **방향**
- 2차 모멘트 (v): gradient 제곱의 이동 평균 → **크기 조절**
- Bias Correction: 초기 단계에서 0에 편향되는 것 보정

### 수식
$$m_t = \beta_1 \cdot m_{t-1} + (1-\beta_1) \cdot g_t$$
$$v_t = \beta_2 \cdot v_{t-1} + (1-\beta_2) \cdot g_t^2$$
$$\hat{m}_t = \frac{m_t}{1 - \beta_1^t}$$
$$\hat{v}_t = \frac{v_t}{1 - \beta_2^t}$$
$$\theta_{t+1} = \theta_t - \frac{\eta \cdot \hat{m}_t}{\sqrt{\hat{v}_t} + \epsilon}$$

- $\beta_1$: 1차 모멘트 decay (보통 0.9)
- $\beta_2$: 2차 모멘트 decay (보통 0.999)

In [ ]:
# =================================================
# TODO: Adam 구현
# =================================================
#
# 수식:
#   m = β₁ * m + (1-β₁) * grads        (1차 모멘트)
#   v = β₂ * v + (1-β₂) * grads²       (2차 모멘트)
#   m_hat = m / (1 - β₁^t)             (bias correction)
#   v_hat = v / (1 - β₂^t)
#   θ_new = θ - lr * m_hat / (√v_hat + ε)
#
# 요구사항:
#   - __init__: lr, beta1, beta2, epsilon 저장
#               m, v는 None, t=0으로 초기화
#   - update: t 증가, m/v 업데이트, bias correction 적용
#
# =================================================

class Adam:
    def __init__(self, lr=0.01, beta1=0.9, beta2=0.999, epsilon=1e-8):
        # TODO: 파라미터들 저장
        # m, v = None, t = 0
        pass
    
    def update(self, params, grads):
        # TODO:
        # 1. m, v가 None이면 초기화
        # 2. t += 1
        # 3. m, v 업데이트
        # 4. bias correction (m_hat, v_hat)
        # 5. params 업데이트
        pass

In [ ]:
# ============ 테스트 ============
def test_adam():
    opt = Adam(lr=0.1, beta1=0.9, beta2=0.999)
    params = np.array([1.0, 2.0])
    grads = np.array([0.5, 0.5])
    
    # 여러 번 업데이트
    for i in range(3):
        params = opt.update(params, grads)
        print(f"Step {i+1}: {params}")
    
    assert params is not None, "update 메서드 구현 필요"
    print("✅ Adam 테스트 통과!")

test_adam()

---
## 6. 시각화 및 비교 (제공됨)

구현한 optimizer들을 비교해봅시다.

In [ ]:
def optimize(func, grad_func, optimizer, start_point, n_iterations=100):
    """최적화 실행 및 궤적 반환"""
    point = np.array(start_point, dtype=float)
    trajectory = [point.copy()]
    losses = [func(point[0], point[1])]
    
    for _ in range(n_iterations):
        grad = grad_func(point[0], point[1])
        point = optimizer.update(point, grad)
        trajectory.append(point.copy())
        losses.append(func(point[0], point[1]))
    
    return np.array(trajectory), np.array(losses)


def plot_comparison(func, trajectories_dict, xlim=(-4, 4), ylim=(-4, 4), title="Optimizer 비교"):
    """여러 optimizer의 궤적 비교"""
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # 등고선
    x = np.linspace(xlim[0], xlim[1], 100)
    y = np.linspace(ylim[0], ylim[1], 100)
    X, Y = np.meshgrid(x, y)
    Z = np.log10(func(X, Y) + 1)
    
    axes[0].contour(X, Y, Z, levels=30, cmap='viridis', alpha=0.5)
    axes[0].contourf(X, Y, Z, levels=30, cmap='viridis', alpha=0.3)
    
    colors = ['red', 'blue', 'green', 'orange']
    for (name, (traj, losses)), color in zip(trajectories_dict.items(), colors):
        axes[0].plot(traj[:, 0], traj[:, 1], 'o-', color=color,
                    label=name, markersize=3, linewidth=1.5)
        axes[0].scatter(traj[0, 0], traj[0, 1], color=color, s=100, marker='*')
        
        axes[1].plot(losses, color=color, label=name, linewidth=2)
    
    axes[0].set_xlabel('x')
    axes[0].set_ylabel('y')
    axes[0].set_title(f'{title} - 궤적')
    axes[0].legend()
    axes[0].set_xlim(xlim)
    axes[0].set_ylim(ylim)
    
    axes[1].set_xlabel('Iteration')
    axes[1].set_ylabel('Loss (log scale)')
    axes[1].set_yscale('log')
    axes[1].set_title(f'{title} - Loss 변화')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# 이차 함수에서 비교
start = [-3.5, 3.5]
n_iter = 50

results = {
    'Vanilla GD': optimize(quadratic, quadratic_gradient, VanillaGD(lr=0.1), start, n_iter),
    'Momentum': optimize(quadratic, quadratic_gradient, MomentumGD(lr=0.1, momentum=0.9), start, n_iter),
    'RMSprop': optimize(quadratic, quadratic_gradient, RMSprop(lr=0.1), start, n_iter),
    'Adam': optimize(quadratic, quadratic_gradient, Adam(lr=0.1), start, n_iter),
}

plot_comparison(quadratic, results, title="이차 함수 (x² + 5y²)")

In [ ]:
# Rosenbrock 함수에서 비교 (더 어려운 케이스)
start = [-1.5, 1.5]
n_iter = 300

results = {
    'Vanilla GD': optimize(rosenbrock, rosenbrock_gradient, VanillaGD(lr=0.001), start, n_iter),
    'Momentum': optimize(rosenbrock, rosenbrock_gradient, MomentumGD(lr=0.001, momentum=0.9), start, n_iter),
    'RMSprop': optimize(rosenbrock, rosenbrock_gradient, RMSprop(lr=0.01), start, n_iter),
    'Adam': optimize(rosenbrock, rosenbrock_gradient, Adam(lr=0.01), start, n_iter),
}

plot_comparison(rosenbrock, results, xlim=(-2, 2), ylim=(-1, 3), title="Rosenbrock 함수")

---
## 7. Post-Quiz (시각화 결과를 보고 답해보세요)

### Q1. 이차 함수에서 Vanilla GD가 지그재그로 움직이는 이유는?
```
당신의 답: 
```

### Q2. Momentum이 지그재그를 줄이는 원리는?
```
당신의 답: 
```

### Q3. Rosenbrock 함수에서 Adam이 가장 잘 된 이유는?
```
당신의 답: 
```

### Q4. 실무에서 Adam을 기본으로 쓰는 이유는?
```
당신의 답: 
```

---
## 정답 및 해설

<details>
<summary>👉 클릭하여 정답 보기</summary>

### Pre-Quiz 정답

**Q1. Gradient Descent 한 문장 정의**
> 손실 함수의 gradient(기울기) 반대 방향으로 파라미터를 조금씩 이동시켜 최솟값을 찾는 최적화 알고리즘

**Q2. Momentum이 필요한 이유**
> - 지그재그 현상 감소 (수직 방향 진동 상쇄)
> - Local minima 탈출 가능성 증가
> - 수렴 속도 향상

**Q3. Adam의 구성**
> - Momentum (1차 모멘트): gradient의 **방향** 정보 (어디로 갈지)
> - RMSprop (2차 모멘트): gradient의 **크기** 정보 (얼마나 갈지)

---

### Post-Quiz 정답

**Q1. 지그재그 이유**
> - y 방향 gradient가 x 방향보다 5배 큼 (5y² vs x²)
> - 같은 lr 적용 시 y 방향으로 과하게 이동
> - 다음 스텝에서 반대로 보정 → 지그재그

**Q2. Momentum 원리**
> - 과거 velocity를 기억
> - 수직(지그재그) 방향: 반대 방향이 상쇄됨
> - 수평(목표) 방향: 같은 방향이 누적됨

**Q3. Adam이 잘 되는 이유**
> - Momentum: 좁은 골짜기에서 방향 유지
> - RMSprop: 방향별로 적응적 lr 조절
> - 두 장점 결합 → 복잡한 지형에서도 효과적

**Q4. Adam을 기본으로 쓰는 이유**
> - lr에 덜 민감 (대부분 0.001로 시작해도 OK)
> - 대부분의 문제에서 안정적
> - 별도 튜닝 없이 좋은 성능

</details>

---
## 구현 정답 코드

<details>
<summary>👉 클릭하여 정답 코드 보기</summary>

```python
class VanillaGD:
    def __init__(self, lr=0.01):
        self.lr = lr
    
    def update(self, params, grads):
        return params - self.lr * grads


class MomentumGD:
    def __init__(self, lr=0.01, momentum=0.9):
        self.lr = lr
        self.momentum = momentum
        self.velocity = None
    
    def update(self, params, grads):
        if self.velocity is None:
            self.velocity = np.zeros_like(params)
        
        self.velocity = self.momentum * self.velocity + self.lr * grads
        return params - self.velocity


class RMSprop:
    def __init__(self, lr=0.01, decay=0.99, epsilon=1e-8):
        self.lr = lr
        self.decay = decay
        self.epsilon = epsilon
        self.cache = None
    
    def update(self, params, grads):
        if self.cache is None:
            self.cache = np.zeros_like(params)
        
        self.cache = self.decay * self.cache + (1 - self.decay) * grads**2
        return params - self.lr * grads / (np.sqrt(self.cache) + self.epsilon)


class Adam:
    def __init__(self, lr=0.01, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.m = None
        self.v = None
        self.t = 0
    
    def update(self, params, grads):
        if self.m is None:
            self.m = np.zeros_like(params)
            self.v = np.zeros_like(params)
        
        self.t += 1
        self.m = self.beta1 * self.m + (1 - self.beta1) * grads
        self.v = self.beta2 * self.v + (1 - self.beta2) * grads**2
        
        m_hat = self.m / (1 - self.beta1**self.t)
        v_hat = self.v / (1 - self.beta2**self.t)
        
        return params - self.lr * m_hat / (np.sqrt(v_hat) + self.epsilon)
```

</details>